# ***projekt numerik***

In [2]:
#%matplotlib widget
import numpy as np
from matplotlib import pyplot as plt
import constants as cst

## ***Implicit model***

In [56]:
# delta's discretisation

delta_t = 0.5
delta_r = 0.01
n = 4

def t_i(i):
    return i*delta_t

def r_j(j):
    return j*delta_r

#T_i_plus_one = np.zeros((n-1,n-1))


##  Constants and their implementations



In [57]:
#Dichte nach Radius
def rho(dist):

    #dichte;

    if(0.0 <=dist <= 0.025):
        return float(cst.WASSER[0])

    if((0.025 < dist <= 0.030) or (0.055< dist) ):
        return float(cst.STAHL[0])

    if(0.030 < dist <= 0.055):
        return float(cst.GLASSWOLLE[0])


# Wärmekapazität nach Radius
def c_p(dist):

    #cap;

    if(0.0 <= dist <= 0.025):
        return float(cst.WASSER[1])

    if((0.025 < dist <= 0.030) or (0.055 < dist) ):
        return float(cst.STAHL[1])

    if(0.030 < dist <= 0.055):
        return float(cst.GLASSWOLLE[1])




#Wärmeleitungskonstante nach Radius
def Lambda(dist):

    

    if(0.0 <= dist <= 0.025):
        return float(cst.WASSER[2])

    if((0.025 < dist <= 0.030) or (0.055< dist) ):
        return  float(cst.STAHL[2])

    if(0.030 < dist <= 0.055):
        return  float(cst.GLASSWOLLE[2])



## Definition of the function of time and its implementation



In [58]:
Gt=20
n = int(cst.size_rhor/delta_r)
time =int(100 / delta_t)

## explicit

$T_{i+1,j} = u_j \frac{r_j}{\Delta r^2}T_{i,j-1} + [1-u_j(\frac{1}{\Delta r} + \frac{2r_j}{\Delta r^2})]T_{i,j} + u_j(\frac{1}{\Delta r}+ \frac{r_j}{\Delta r^2})T_{i,j+1}$


for $u_j = \frac{\Delta t}{r_j}k_j$ and $k_j = \frac{\lambda _j}{Cp_{j} \rho _j}$

In [5]:


T = np.zeros([time+1,n+1])
T[0, :] = 0
T[0, n] = Gt

print(T[0,:])

for i in range(0,time):

    T[i][n] = Gt

    for j in range(1,n):

        r = r_j(j)
        k_j = Lambda(r) / (c_p(r)*rho(r))
        u_j = (delta_t*k_j) / r

        A = T[i][j-1]
        B = T[i][j]
        C = T[i][j+1]
    
        T[i+1][j] = u_j*r/(delta_r**2)*A + (1-u_j*(1/delta_r + 2*r/delta_r**2))*B + u_j*(1/delta_r + r/delta_r**2)*C

    T[i+1, 0] = T[i+1, 1]  
    T[i+1, n] = Gt


[ 0.  0.  0.  0.  0.  0. 20.]


(201, 7) (201, 7)


In [6]:
from matplotlib import cm
# Graphische Darstellung
x = np.array(range(n+1))*delta_r
t = np.array(range(time+1))*delta_t

xA, tA = np.meshgrid(x, t)
print(tA.shape, xA.shape)

# Plot the surface.
fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, num=1)
surf = ax.plot_surface(xA, tA, T,cmap=cm.coolwarm)
fig.colorbar(surf, shrink=0.5, aspect=5)

In [7]:
import math as mt
T = np.zeros([time+1,n+1])
T[0, :] = 0
T[0, n] = 0

print(T[0,:])

for i in range(0,time):

    T[i][n] = 20*mt.sin(2*3.14159265*i)

    for j in range(1,n):

        r = r_j(j)
        k_j = Lambda(r) / (c_p(r)*rho(r))
        u_j = (delta_t*k_j) / r

        A = T[i][j-1]
        B = T[i][j]
        C = T[i][j+1]
    
        T[i+1][j] = u_j*r/(delta_r**2)*A + (1-u_j*(1/delta_r + 2*r/delta_r**2))*B + u_j*(1/delta_r + r/delta_r**2)*C

    T[i+1, 0] = T[i+1, 1]  
    T[i+1, n] = 20*mt.sin(2*3.14159265*(i))

[0. 0. 0. 0. 0. 0. 0.]


(201, 7) (201, 7)


In [8]:
from matplotlib import cm
# Graphische Darstellung
x = np.array(range(n+1))*delta_r
t = np.array(range(time+1))*delta_t

xA, tA = np.meshgrid(x, t)
print(tA.shape, xA.shape)

# Plot the surface.
fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, num=1)
surf = ax.plot_surface(xA, tA, T,cmap=cm.coolwarm)
fig.colorbar(surf, shrink=0.5, aspect=5)

## implicit

$-w_j T_{i+1,j-1} + (1+v_j+2w_j) T_{i+1,j}-(v_j+w_j)T_{i+1,j+1}=T_{i,j}$

With $v_j = \frac{u_j}{\Delta r}$ , $w_j=2r_j \frac{u_j}{\Delta r^2}$ and $u_j = \frac{\lambda_j \Delta t}{r_j C_{p,j} \rho_j}$

In [11]:
T = np.zeros([time+1,n+1])
T[0, :] = 0

R = r_j(n-1)
Ujboundary = (Lambda(R) * delta_t)/(R*rho(R)*c_p(R))
Vjboundary = Ujboundary/delta_r
Wjboundary = (2*R*Ujboundary)/(delta_r**2)

T[0, n] = Gt*(Ujboundary+Wjboundary)

# I want to create the matrix of coefficients A (which holds the coefficients for the vector of Ti+1,j) and I want to create the vector B (which holds the values of the Ti,j)
#then at each temporal increment, I'm solving the eqaution A T_i+1 = B and updating the values at T_i+1 with the results

for i in range(0,time):

    T[i][n] = Gt*(Ujboundary+Wjboundary)

    A = np.zeros([time+1,n+1])
    # time-1 x n-1 matrix, Matrix of coefficient
    B = T[i,:]
    
    for j in range(1,n):

        r = r_j(j)
        Uj = (Lambda(r) * delta_t)/(r*rho(r)*c_p(r))
        Vj = Uj/delta_r
        Wj = (2*r*Uj)/(delta_r**2)

        # testing for the boudary condition at T'(t,0)=0 fro the first element of the matrix
        if j==1:
            A[1,1]=1+Vj+Wj
            A[i+1,j+1]= -Vj-Wj
        else:
            A[i+1,j-1] = -Wj
            A[i+1,j] = 1+Vj+2*Wj
            A[i+1,j+1] = -Vj-Wj
    

    T[i+1,:] = np.linalg.solve(A,B)    
        
        
        

LinAlgError: Last 2 dimensions of the array must be square

In [89]:
from scipy.linalg import lu_factor, lu_solve, lu

A = np.zeros([n-1,n-1])

for j in range (0,n-1):

    r = r_j(j+1)
    Uj = (Lambda(r) * delta_t)/(r*rho(r)*c_p(r))
    Vj = Uj/delta_r
    Wj = (2*r*Uj)/(delta_r**2)

    A[j,j] = 1+Vj+2*Wj
#first coeff.
    if j == 0:
        A[j,j]= 1+Vj+Wj
#upper diagonal
    if (j+1) <= (n-2):
        A[j,j+1] = -(Vj+Wj)
#lower diagonal  
    if j-1 >= 0:
        A[j,j-1] = -Wj

print(A)
print(A[n-2,n-2])
lu_A, P = lu_factor(A)
print(lu_A,P)

T = np.zeros([time,n-1])
T[0, :] = 0
T[:,n-2] = Gt #boundary conditions at R. Here we use n-1 since the iteration goes from 0 to n-1
b=np.zeros([n-1])

print(T[0,:].shape,b.shape)

for i in range (0,time):
    b = T[i,:]
    T[i+1,:] = lu_solve((lu_A,P),b)




[[ 1.00214459 -0.00214459  0.          0.          0.        ]
 [-0.00142972  1.00321688 -0.00178715  0.          0.        ]
 [ 0.         -0.04079161  1.08838182 -0.04759021  0.        ]
 [ 0.          0.         -0.00651042  1.01383464 -0.00732422]
 [ 0.          0.          0.         -0.00651042  1.01367188]]
1.013671875
[[ 1.00214459 -0.00214459  0.          0.          0.        ]
 [-0.00142666  1.00321382 -0.00178715  0.          0.        ]
 [ 0.         -0.04066093  1.08830915 -0.04759021  0.        ]
 [ 0.          0.         -0.00598214  1.01354994 -0.00732422]
 [ 0.          0.          0.         -0.00642338  1.01362483]] [0 1 2 3 4]
(5,) (5,)


IndexError: index 200 is out of bounds for axis 0 with size 200